In [1]:
import numpy as np
import pandas as pd
import requests

import configparser
config = configparser.ConfigParser()
config.read('../config.ini')

import geopandas as gpd
from shapely.geometry import LineString

In [2]:
df_bike_sample = pd.read_csv('../data/bike_rental_history/processed/2022_weekday_sample.csv')
gdf_bike_station = gpd.read_file('../data/station/station.geojson')

In [12]:
df_bike_sample.loc[:,'route'].unique().tolist()[:9000]

['ST-2891~ST-99',
 'ST-1751~ST-86',
 'ST-2046~ST-1689',
 'ST-1599~ST-1344',
 'ST-1474~ST-2718',
 'ST-2001~ST-1994',
 'ST-273~ST-271',
 'ST-834~ST-1249',
 'ST-2037~ST-834',
 'ST-102~ST-259',
 'ST-1632~ST-251',
 'ST-547~ST-548',
 'ST-891~ST-739',
 'ST-2547~ST-984',
 'ST-2074~ST-600',
 'ST-1712~ST-2031',
 'ST-1506~ST-3068',
 'ST-2549~ST-2791',
 'ST-1173~ST-1174',
 'ST-2112~ST-2549',
 'ST-99~ST-2545',
 'ST-1916~ST-753',
 'ST-362~ST-375',
 'ST-286~ST-1543',
 'ST-1014~ST-316',
 'ST-3072~ST-3142',
 'ST-416~ST-311',
 'ST-384~ST-97',
 'ST-1880~ST-2668',
 'ST-5~ST-211',
 'ST-1035~ST-1033',
 'ST-2290~ST-847',
 'ST-1061~ST-495',
 'ST-735~ST-2067',
 'ST-1592~ST-1869',
 'ST-2395~ST-2960',
 'ST-1990~ST-2821',
 'ST-1753~ST-1238',
 'ST-2490~ST-2943',
 'ST-664~ST-873',
 'ST-1250~ST-2481',
 'ST-602~ST-1123',
 'ST-1916~ST-752',
 'ST-398~ST-2501',
 'ST-1793~ST-1729',
 'ST-1684~ST-1083',
 'ST-1578~ST-2984',
 'ST-403~ST-2504',
 'ST-243~ST-2342',
 'ST-2037~ST-1251',
 'ST-738~ST-896',
 'ST-513~ST-2482',
 'ST-4

In [13]:
url = "https://apis-navi.kakaomobility.com/v1/directions?origin={},{}&destination={},{}&avoid=motorway"
api_key = config['kakao']['key']
headers = {"Authorization": "KakaoAK " + api_key}
list_path = []
i=0
for route in df_bike_sample.loc[:,'route'].unique().tolist()[:9000]:
    start_station, end_station = route.split('~')
    start_x, start_y = gdf_bike_station.loc[gdf_bike_station.loc[:,'대여소_ID']==start_station,['경도','위도']].values[0]
    end_x, end_y = gdf_bike_station.loc[gdf_bike_station.loc[:,'대여소_ID']==end_station,['경도','위도']].values[0]
    url_format = url.format(start_x, start_y, end_x, end_y)

    result = requests.get(url_format, headers=headers).json()
    try:
        nested_path = [i['vertexes'] for i in result['routes'][0]['sections'][0]['roads']]
        flatten_path = [coord for sublist in nested_path for coord in sublist]
        path = [[flatten_path[i], flatten_path[i+1]] for i in range(0, len(flatten_path), 2)]
    except:
        print(result)
        path = None
    list_path.append({'route':route, 'path':path})
    i=i+1
    if(i%100==0):
        print(i)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
{'trans_id': '018e4ed88ea17ede8f6e87a85e73ffaf', 'routes': [{'result_code': 1, 'result_msg': '길찾기 결과를 찾을 수 없음'}]}
2600
{'trans_id': '018e4ed910fe7e3881e1077be7310481', 'routes': [{'result_code': 1, 'result_msg': '길찾기 결과를 찾을 수 없음'}]}
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
{'trans_id': '018e4ee16de376e7a92c4ae316c84084', 'routes': [{'result_code': 1, 'result_msg': '길찾기 결과를 찾을 수 없음'}]}
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
{'trans_id': '018e4eead6ea7d1698bd9672166f544a', 'routes': [{'result_code': 1, 'result_msg': '길찾기 결과를 찾을 수 없음'}]}
6500
6600
6700
6800
6900
7000
7100
7200
{'trans_id': '018e4eee58b7744ab8f478dd6c23865e', 'routes': [{'result_code': 1, 'result_msg': '길찾기 결과를 찾을 수 없음'}]}
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
{'trans_id': '018e4ef38d

In [18]:
df_routes = pd.DataFrame(list_path)
df_routes = df_routes.loc[np.logical_not(df_routes.loc[:,'path'].isnull())]
df_bike_sample_first_trial = df_bike_sample.loc[df_bike_sample.loc[:,'route'].isin(df_routes.loc[:,'route'])]
df_bike_sample_first_trial = df_bike_sample_first_trial.merge(df_routes,
                                                              on='route').loc[:,['대여일시',
                                                                                 '반납일시',
                                                                                 '대여대여소ID',
                                                                                 '반납대여소ID',
                                                                                 'path']].rename(columns={'대여일시':'rent_datetime',
                                                                                                          '반납일시':'return_datetime',
                                                                                                          '대여대여소ID':'rent_at',
                                                                                                          '반납대여소ID':'return_at'})

In [21]:
df_bike_sample_first_trial.loc[:,'rent_datetime'] = pd.to_datetime(df_bike_sample_first_trial.loc[:,'rent_datetime'])
df_bike_sample_first_trial.loc[:,'return_datetime'] = pd.to_datetime(df_bike_sample_first_trial.loc[:,'return_datetime'])


list_timestamp = []
for i, row in df_bike_sample_first_trial.iterrows():
    rent_hour = row['rent_datetime'].hour
    rent_minute = row['rent_datetime'].minute
    rent_second = row['rent_datetime'].second
    rent_timestamp = rent_hour*60*60 + rent_minute*60 +rent_second

    return_hour = row['return_datetime'].hour
    return_minute = row['return_datetime'].minute
    return_second = row['return_datetime'].second
    return_timestamp = return_hour*60*60 + return_minute*60 +return_second

    list_timestamp.append([rent_timestamp, return_timestamp])

df_bike_sample_first_trial.loc[:,'timestamp'] = list_timestamp

C:\Users\seung\AppData\Local\Temp\ipykernel_13508\3229010693.py:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_bike_sample_first_trial.loc[:,'rent_datetime'] = pd.to_datetime(df_bike_sample_first_trial.loc[:,'rent_datetime'])
C:\Users\seung\AppData\Local\Temp\ipykernel_13508\3229010693.py:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_bike_sample_first_trial.loc[:,'return_datetime'] = pd.to_datetime(df_bike_sample_first_trial.loc[:,'return_datetime'])


In [23]:
list_timestamp = []
for i, row in df_bike_sample_first_trial.iterrows():
    distance = LineString(row['path']).length
    start_time, end_time = row['timestamp']
    velocity = distance / (end_time - start_time)
    path = row['path']
    timestamp = []
    timestamp.append(start_time)
    current_time = start_time
    for i in range(0,len(path)-2):
        segment_distance = LineString([path[i], path[i+1]]).length
        new_time = int(current_time + segment_distance/velocity)
        timestamp.append(new_time)
        current_time = new_time
    timestamp.append(end_time)
    list_timestamp.append(','.join([str(t) for t in timestamp]))

df_bike_sample_first_trial.loc[:,'timestamp'] = list_timestamp
df_bike_sample_first_trial.loc[:,'timestamp'] = df_bike_sample_first_trial.loc[:,'timestamp'].apply(lambda x: [int(n) for n in x.split(',')])

In [24]:
df_bike_sample_first_trial

,rent_datetime,return_datetime,rent_at,return_at,path,timestamp
0,2022-07-29 09:00:15,2022-07-29 09:08:30,ST-2891,ST-99,"[[127.08485622273864, 37.52889920173301], [127...","[32415, 32427, 32427, 32443, 32449, 32465, 324..."
1,2022-12-08 08:29:20,2022-12-08 08:35:21,ST-2891,ST-99,"[[127.08485622273864, 37.52889920173301], [127...","[30560, 30569, 30569, 30580, 30584, 30596, 306..."
2,2022-12-06 08:27:56,2022-12-06 08:34:28,ST-2891,ST-99,"[[127.08485622273864, 37.52889920173301], [127...","[30476, 30485, 30485, 30497, 30502, 30515, 305..."
3,2022-12-15 08:31:01,2022-12-15 08:36:54,ST-2891,ST-99,"[[127.08485622273864, 37.52889920173301], [127...","[30661, 30669, 30669, 30680, 30684, 30695, 307..."
4,2022-11-16 06:27:36,2022-11-16 06:39:39,ST-2891,ST-99,"[[127.08485622273864, 37.52889920173301], [127...","[23256, 23274, 23274, 23297, 23306, 23330, 233..."
...,...,...,...,...,...,...
37023,2022-08-02 11:29:48,2022-08-02 11:40:02,ST-1690,ST-446,"[[127.0658773136203, 37.54112205726897], [127....","[41388, 41388, 41388, 41395, 41395, 41397, 414..."
37024,2022-12-14 17:15:11,2022-12-14 17:19:12,ST-2596,ST-2594,"[[127.1036311788207, 37.5161851182297], [127.1...","[62111, 62139, 62143, 62167, 62178, 62190, 622..."
37025,2022-11-21 21:51:53,2022-11-21 21:56:04,ST-2596,ST-2594,"[[127.1036311788207, 37.5161851182297], [127.1...","[78713, 78743, 78747, 78772, 78783, 78796, 788..."
37026,2022-07-08 00:14:35,2022-07-08 00:17:52,ST-2596,ST-2594,"[[127.1036311788207, 37.5161851182297], [127.1...","[875, 898, 901, 920, 929, 939, 954, 958, 970, ..."


In [28]:
gdf_bike_station = gdf_bike_station.rename(columns={'대여소_ID':'id','경도':'x','위도':'y'})
df_bike_sample_first_trial = df_bike_sample_first_trial.merge(gdf_bike_station.loc[:,['id','x','y']], left_on='rent_at', right_on='id').rename(columns={'x':'rent_x','y':'rent_y'})
df_bike_sample_first_trial = df_bike_sample_first_trial.merge(gdf_bike_station.loc[:,['id','x','y']], left_on='return_at', right_on='id').rename(columns={'x':'return_x','y':'return_y'})

In [30]:
df_bike_sample_first_trial = df_bike_sample_first_trial.drop(['id_x','id_y'], axis=1)

In [32]:
df_bike_sample_first_trial.to_json('../data/bike_rental_history/processed/sample_1st.json', orient='records')

이 부분은 계속해서 Route 수집하기 위한 용도로 쓰여진 코드

In [3]:
df_bike_sample = pd.read_csv('../data/bike_rental_history/processed/2022_weekday_sample.csv')
gdf_bike_station = gpd.read_file('../data/station/station.geojson')

In [4]:
df_gathered = pd.read_json('../data/bike_rental_history/processed/sample_1_2.json')

In [6]:
df_gathered.loc[:,'route'] = df_gathered.loc[:,'rent_at'].astype(str) + '~' + df_gathered.loc[:,'return_at'].astype(str)

In [7]:
df_bike_sample = df_bike_sample.loc[np.logical_not(df_bike_sample.loc[:,'route'].isin(df_gathered.loc[:,'route']))]

In [10]:
url = "https://apis-navi.kakaomobility.com/v1/directions?origin={},{}&destination={},{}&avoid=motorway"
api_key = config['kakao']['key']
headers = {"Authorization": "KakaoAK " + api_key}
list_path = []
i=0
for route in df_bike_sample.loc[:,'route'].unique().tolist():
    start_station, end_station = route.split('~')
    start_x, start_y = gdf_bike_station.loc[gdf_bike_station.loc[:,'대여소_ID']==start_station,['경도','위도']].values[0]
    end_x, end_y = gdf_bike_station.loc[gdf_bike_station.loc[:,'대여소_ID']==end_station,['경도','위도']].values[0]
    url_format = url.format(start_x, start_y, end_x, end_y)

    result = requests.get(url_format, headers=headers).json()
    try:
        nested_path = [i['vertexes'] for i in result['routes'][0]['sections'][0]['roads']]
        flatten_path = [coord for sublist in nested_path for coord in sublist]
        path = [[flatten_path[i], flatten_path[i+1]] for i in range(0, len(flatten_path), 2)]
    except:
        print(result)
        path = None
    list_path.append({'route':route, 'path':path})
    i=i+1
    if(i%100==0):
        print(i)



{'trans_id': '018e75d6f10873919d094a1d081154cc', 'routes': [{'result_code': 1, 'result_msg': '길찾기 결과를 찾을 수 없음'}]}
{'trans_id': '018e75d6f8b97ab59dff344d76c6d6eb', 'routes': [{'result_code': 1, 'result_msg': '길찾기 결과를 찾을 수 없음'}]}
{'trans_id': '018e75d703c776b189962ed4fd9961fd', 'routes': [{'result_code': 1, 'result_msg': '길찾기 결과를 찾을 수 없음'}]}
{'trans_id': '018e75d70e947771a6dd255d1be3627a', 'routes': [{'result_code': 1, 'result_msg': '길찾기 결과를 찾을 수 없음'}]}
{'trans_id': '018e75d718477f7284e2955d458ab12a', 'routes': [{'result_code': 1, 'result_msg': '길찾기 결과를 찾을 수 없음'}]}
{'trans_id': '018e75d722967fa2a5d340ff3d837730', 'routes': [{'result_code': 1, 'result_msg': '길찾기 결과를 찾을 수 없음'}]}
{'trans_id': '018e75d72dc37fd79baa6e3cada5f13f', 'routes': [{'result_code': 1, 'result_msg': '길찾기 결과를 찾을 수 없음'}]}
{'trans_id': '018e75d738ab7b748a3804ec90f48c9e', 'routes': [{'result_code': 106, 'result_msg': '도착 지점 주변의 도로에 유고 정보(교통 장애)가 있음'}]}
{'trans_id': '018e75d739a878e8878aea7590e8e642', 'routes': [{'result_co

In [11]:
df_routes = pd.DataFrame(list_path)
df_routes = df_routes.loc[np.logical_not(df_routes.loc[:,'path'].isnull())]
df_bike_sample_first_trial = df_bike_sample.loc[df_bike_sample.loc[:,'route'].isin(df_routes.loc[:,'route'])]
df_bike_sample_first_trial = df_bike_sample_first_trial.merge(df_routes,
                                                              on='route').loc[:,['대여일시',
                                                                                 '반납일시',
                                                                                 '대여대여소ID',
                                                                                 '반납대여소ID',
                                                                                 'path']].rename(columns={'대여일시':'rent_datetime',
                                                                                                          '반납일시':'return_datetime',
                                                                                                          '대여대여소ID':'rent_at',
                                                                                                          '반납대여소ID':'return_at'})

df_bike_sample_first_trial.loc[:,'rent_datetime'] = pd.to_datetime(df_bike_sample_first_trial.loc[:,'rent_datetime'])
df_bike_sample_first_trial.loc[:,'return_datetime'] = pd.to_datetime(df_bike_sample_first_trial.loc[:,'return_datetime'])


list_timestamp = []
for i, row in df_bike_sample_first_trial.iterrows():
    rent_hour = row['rent_datetime'].hour
    rent_minute = row['rent_datetime'].minute
    rent_second = row['rent_datetime'].second
    rent_timestamp = rent_hour*60*60 + rent_minute*60 +rent_second

    return_hour = row['return_datetime'].hour
    return_minute = row['return_datetime'].minute
    return_second = row['return_datetime'].second
    return_timestamp = return_hour*60*60 + return_minute*60 +return_second

    list_timestamp.append([rent_timestamp, return_timestamp])

df_bike_sample_first_trial.loc[:,'timestamp'] = list_timestamp

list_timestamp = []
for i, row in df_bike_sample_first_trial.iterrows():
    distance = LineString(row['path']).length
    start_time, end_time = row['timestamp']
    velocity = distance / (end_time - start_time)
    path = row['path']
    timestamp = []
    timestamp.append(start_time)
    current_time = start_time
    for i in range(0,len(path)-2):
        segment_distance = LineString([path[i], path[i+1]]).length
        new_time = int(current_time + segment_distance/velocity)
        timestamp.append(new_time)
        current_time = new_time
    timestamp.append(end_time)
    list_timestamp.append(','.join([str(t) for t in timestamp]))

df_bike_sample_first_trial.loc[:,'timestamp'] = list_timestamp
df_bike_sample_first_trial.loc[:,'timestamp'] = df_bike_sample_first_trial.loc[:,'timestamp'].apply(lambda x: [int(n) for n in x.split(',')])

gdf_bike_station = gdf_bike_station.rename(columns={'대여소_ID':'id','경도':'x','위도':'y'})
df_bike_sample_first_trial = df_bike_sample_first_trial.merge(gdf_bike_station.loc[:,['id','x','y']], left_on='rent_at', right_on='id').rename(columns={'x':'rent_x','y':'rent_y'})
df_bike_sample_first_trial = df_bike_sample_first_trial.merge(gdf_bike_station.loc[:,['id','x','y']], left_on='return_at', right_on='id').rename(columns={'x':'return_x','y':'return_y'})

C:\Users\seung\AppData\Local\Temp\ipykernel_9828\3805954960.py:14: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_bike_sample_first_trial.loc[:,'rent_datetime'] = pd.to_datetime(df_bike_sample_first_trial.loc[:,'rent_datetime'])
C:\Users\seung\AppData\Local\Temp\ipykernel_9828\3805954960.py:15: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_bike_sample_first_trial.loc[:,'return_datetime'] = pd.to_datetime(df_bike_sample_first_trial.loc[:,'return_datetime'])


In [12]:
df_bike_sample_first_trial = df_bike_sample_first_trial.drop(['id_x','id_y'], axis=1)

In [13]:
df_bike_sample_first_trial.to_json('../data/bike_rental_history/processed/sample_3nd.json', orient='records')